In [ ]:
!pip install fs

In [3]:
import pandas as pd
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

## TESTING HYBRID

In [3]:
df = pd.read_csv("./DataSources/M1_OptimizeFullReport.csv", encoding='latin1')
merged_df = pd.read_csv("./DataSources/Dataset_V1.csv")

In [5]:
import pandas as pd
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("./DataSources/M1_OptimizeFullReport.csv", encoding='latin1')
merged_df = pd.read_csv("./DataSources/Dataset_V1.csv")

# Calculate user-user similarity based on RFAT and wallet share by category
def calculate_user_similarity(merged_df):
    user_features = merged_df.drop(columns=['UID', 'Conversion Time'])  # Exclude UID and Conversion Time
    user_similarity_matrix = cosine_similarity(user_features)
    return user_similarity_matrix

# Example: Get top N similar users for a given user
def get_top_similar_users(merged_df, user_similarity_matrix, user_id, n=5):
    if user_id not in merged_df['UID'].values:
        print(f"User ID {user_id} does not exist. Using default recommendations.")
        return []
    user_index = merged_df.index[merged_df['UID'] == user_id][0]
    similar_users = user_similarity_matrix[user_index].argsort()[::-1][1:n+1]
    return similar_users

# Example: Aggregate top categories from similar users
def aggregate_top_categories_from_similar_users(merged_df, similar_users, n=10):
    category_columns = [col for col in merged_df.columns if col not in ['UID', 'Conversion Time', 'Recency', 'Frequency', 'AOV', 'Total Spending','Unique Category Counts']]
    top_categories = merged_df.iloc[similar_users][category_columns].sum().nlargest(n).index.tolist()
    return top_categories

# Example: Recommend top categories for a given user
def recommend_top_categories(merged_df, user_similarity_matrix, user_id, n=10):
    similar_users = get_top_similar_users(merged_df, user_similarity_matrix, user_id)
    if len(similar_users) == 0:  # Check if no similar users were found
        return default_recommendations(merged_df), []  # Fallback to default recommendations and empty offer categories
    top_categories = aggregate_top_categories_from_similar_users(merged_df, similar_users, n)
    return top_categories

# Example: Default recommendations for new users
def default_recommendations(df, n=10):
    # Provide default recommendations based on popular or trending categories
    # Example: Default recommendations for new users
    # Group by category and sum the order value to find the most purchased categories
    top_categories = df.groupby('ex3/category')['Order Value'].sum().nlargest(n).index.tolist()
    return top_categories

# Calculate content-based recommendations based on user history
def recommend_content_based(merged_df, user_id, n=10):
    user_data = merged_df[merged_df['UID'] == user_id]
    if user_data.empty:  # Check if user has no historical data
        return default_recommendations(df, n)  # Fallback to default recommendations
    # Filter out categories with zero values
    user_categories = user_data.drop(columns=['UID', 'Conversion Time', 'Recency', 'Frequency', 'AOV', 'Total Spending','Unique Category Counts']).iloc[0]
    user_categories_nonzero = user_categories[user_categories > 0]
    
    if len(user_categories_nonzero) >= n:
        top_categories = user_categories_nonzero.nlargest(n).index.tolist()
    else:
        #top_categories = user_categories_nonzero.index.tolist()
        top_categories = user_categories_nonzero.sort_values(ascending=False).index.tolist()
    
    return top_categories

# Load the offer dataset from the CSV file
offer_df = pd.read_csv("./DataSources/M1_offers.csv").fillna("");

# Define the mapping between transaction sub-categories and offer main categories
category_mapping = {
    'Beauty': ['Beauty'],
    'Electronics': ['Audio', 'Mobile & Gadgets', 'Cameras & Drones', 'Computers & Accessories', 'Electronics Accessories', 'Monitors & Printers', 'Smart Devices', 'Digital Utilities', 'Televisions & Videos', 'Digital Goods', 'Data Storage', 'Mobiles & Tablets'],
    'Entertainment': ['Gaming & Consoles', 'Media, Music & Books'],
    'Fashion & Retail': ['Fashion Accessories', 'Women Shoes', 'Women Clothes', 'Men Clothes', 'Baby & Kids Fashion', 'Women Bags', 'Muslim Fashion', 'Men Bags', 'Men Shoes', "Women's Shoes and Clothing", "Men's Shoes and Clothing", "Kids' Shoes and Clothing"],
    'Health and Wellness': ['Health'],
    'Hobbies': ['Hobbies & Collections', 'Toys & Games'],
    'Home Improvement/Furniture': ['Home & Living', 'Home Appliances', 'Kitchen & Dining', 'Lighting & Décor', 'Small Appliances', 'Household Supplies', 'Bedding & Bath', 'Outdoor & Garden', 'Furniture & Organization', 'Tools & Home Improvement', 'Laundry & Cleaning Equipment', 'Large Appliances'],
    'Jewelry & Timepieces': ['Watches', 'Watches Sunglasses Jewellery'],
    'Miscellaneous': ['Automobiles', 'Pets', 'Mom & Baby', 'Books & Magazines', 'Stationery', 'Tickets, Vouchers & Services', 'Motorcycles', 'Free Sample (Flexi Combo)', 'Pet Supplies', 'Mother & Baby', 'Stationery, Craft & Gift Cards', 'Motors', 'Surprise Box', 'Telco', 'Stationery & Craft', 'Local Service', 'Services'],
    'Sports and Fitness': ['Sports & Outdoors', 'Sports Shoes and Clothing'],
    'Supermarkets': ['Food & Beverage', 'Groceries'],
    'Travel': ['Travel & Luggage', 'Bags and Travel'],
    'Retailers': [],
    'Marketplace': [],
    'Cafe': [],
    'Finance and Banking': [],
    'Department Stores': [],
    'Visa Generic': [],
    'Real Estate': [],
    'Restaurants': []
}

# Map transaction sub-categories to offer main categories
def map_to_offer_category(sub_categories):
    offer_categories = set()
    for sub_category in sub_categories:
        for offer_category, mapped_sub_categories in category_mapping.items():
            if sub_category in mapped_sub_categories:
                offer_categories.add(offer_category)
    return list(offer_categories)

# Map top categories to offer categories
def map_top_categories_to_offers(top_categories):
    offer_categories = []
    for category in top_categories:
        mapped_category = map_to_offer_category([category])[0]
        if mapped_category not in offer_categories:
            offer_categories.append(mapped_category)
    return offer_categories

# Define function to recommend offers for each offer category
def recommend_offers_for_category(offer_df, offer_category):
    # Filter offers based on the "Merchant_type" column to match the offer category
    matching_offers = offer_df[offer_df['Merchant_type'] == offer_category]
    return matching_offers

# Define function to recommend offers for each offer category
def recommend_offers_for_categories(offer_df, offer_categories):
    recommended_offers = {}
    for category in offer_categories:
        recommended_offers[category] = recommend_offers_for_category(offer_df, category)
    return recommended_offers

# Example: Print top and offer categories for a user
def print_top_and_offer_categories(merged_df, user_similarity_matrix, user_id, offer_df):
    top_categories_content = recommend_content_based(merged_df, user_id)
    if len(top_categories_content) < 10:  # If content-based recommendations are fewer than 10, fill with collaborative filtering
        remaining_categories = 10 - len(top_categories_content)
        top_categories_collaborative = recommend_top_categories(merged_df, user_similarity_matrix, user_id, remaining_categories)
        top_categories_hybrid = top_categories_content + top_categories_collaborative
    else:
        top_categories_hybrid = top_categories_content
    
    print(f"Top categories for user {user_id}:")
    for i, category in enumerate(top_categories_hybrid, start=1):
        print(f"{i}. {category}")

    offer_categories = map_top_categories_to_offers(top_categories_hybrid)

    print("\nOffer categories:")
    for i, category in enumerate(offer_categories, start=1):
        print(f"{i}. {category}")

    recommended_offers = recommend_offers_for_categories(offer_df, offer_categories)

    print(f"\nOffers we recommend for user {user_id}:")
    for i, category in enumerate(offer_categories, start=1):
        print(f"\n{i}. {category}")
        if not recommended_offers[category].empty:
            print(recommended_offers[category][['offer_id', 'merchant_name', 'offer_title', 'Offer_amount(%)']].to_string(index=False))
        else:
            print("No offers available for this category.")


# Example: Get top category recommendations for a user
# Example: Get top category recommendations for a user
#003d363e-5e74-4bd9-9b7f-77d1ddf0ad62 Unique Category Counts 1
#05e3cacf-ce64-488f-9f42-f8c1c0a78fd5 Unique Category Counts 22
#009caacf-c094-4b84-adde-424c584b0b33 Unique Category Counts 3
#06034ff3-ba22-4b1c-9f09-6253c35e7658 Unique Category Counts 4
#01 New User
user_id = '01'
user_similarity_matrix = calculate_user_similarity(merged_df)
print_top_and_offer_categories(merged_df, user_similarity_matrix, user_id, offer_df)


Top categories for user 01:
1. Beauty
2. Home & Living
3. Health
4. Mobile & Gadgets
5. Groceries
6. Food & Beverage
7. Home Appliances
8. Small Appliances
9. Electronics Accessories
10. Mother & Baby

Offer categories:
1. Beauty
2. Home Improvement/Furniture
3. Health and Wellness
4. Electronics
5. Supermarkets
6. Miscellaneous

Offers we recommend for user 01:

1. Beauty
 offer_id merchant_name         offer_title Offer_amount(%)
       51       Sephora       5.5% Cashback             5.5
       51       Sephora Up to 5.5% Cashback                
       51       Sephora Up to 5.5% Cashback                

2. Home Improvement/Furniture
No offers available for this category.

3. Health and Wellness
 offer_id merchant_name   offer_title Offer_amount(%)
       15      Watsons  1.5% Cashback             1.5
       15      Watsons  1.5% Cashback                
       15      Watsons  1.5% Cashback                
       15      Watsons  1.5% Cashback                
       15      Watso

In [7]:
def get_offers_for_user(merged_df, user_similarity_matrix, user_id, offer_df):
    response = {
        'user_id': user_id,
        'top_categories': [],
        'offer_categories': [],
        'recommended_offers': {}
    }
    
    # Get content-based top categories
    top_categories_content = recommend_content_based(merged_df, user_id)
    if len(top_categories_content) < 10:  # If content-based recommendations are fewer than 10, fill with collaborative filtering
        remaining_categories = 10 - len(top_categories_content)
        top_categories_collaborative = recommend_top_categories(merged_df, user_similarity_matrix, user_id, remaining_categories)
        top_categories_hybrid = top_categories_content + top_categories_collaborative
    else:
        top_categories_hybrid = top_categories_content

    response['top_categories'] = top_categories_hybrid
    
    # Map top categories to offer categories
    offer_categories = map_top_categories_to_offers(top_categories_hybrid)
    response['offer_categories'] = offer_categories
    
    # Gather recommended offers for each category
    recommended_offers = recommend_offers_for_categories(offer_df, offer_categories)
    for category in offer_categories:
        if not recommended_offers[category].empty:
            # Extract and include offer details in the response
            offer_details = recommended_offers[category][['offer_id', 'merchant_name', 'offer_title', 'Offer_amount(%)']]
            response['recommended_offers'][category] = offer_details.to_dict(orient='records')
        else:
            response['recommended_offers'][category] = "No offers available for this category."
    
    return response

# Usage example:
user_id = '01'
user_similarity_matrix = calculate_user_similarity(merged_df)
offers = get_offers_for_user(merged_df, user_similarity_matrix, user_id, offer_df)


In [8]:
offers

{'user_id': '01',
 'top_categories': ['Beauty',
  'Home & Living',
  'Health',
  'Mobile & Gadgets',
  'Groceries',
  'Food & Beverage',
  'Home Appliances',
  'Small Appliances',
  'Electronics Accessories',
  'Mother & Baby'],
 'offer_categories': ['Beauty',
  'Home Improvement/Furniture',
  'Health and Wellness',
  'Electronics',
  'Supermarkets',
  'Miscellaneous'],
 'recommended_offers': {'Beauty': [{'offer_id': 51,
    'merchant_name': 'Sephora',
    'offer_title': '5.5% Cashback',
    'Offer_amount(%)': '5.5'},
   {'offer_id': 51,
    'merchant_name': 'Sephora',
    'offer_title': 'Up to 5.5% Cashback',
    'Offer_amount(%)': ''},
   {'offer_id': 51,
    'merchant_name': 'Sephora',
    'offer_title': 'Up to 5.5% Cashback',
    'Offer_amount(%)': ''}],
  'Home Improvement/Furniture': 'No offers available for this category.',
  'Health and Wellness': [{'offer_id': 15,
    'merchant_name': 'Watsons ',
    'offer_title': '1.5% Cashback',
    'Offer_amount(%)': '1.5'},
   {'offer_id'